In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import models
from torchvision.models.vgg import VGG
from PIL import Image
import numpy as np
import os
import nibabel as nib
import time
import datetime
import torch.nn.functional as F
from models.unet import UNet
import matplotlib.pyplot as plt
from metrics.torch_seg_metrics import dice_score, iou
from datasets.BRATS2018 import ToTensorVal, NormalizeBRATSVal, ZeroPadVal
from datasets.BRATS2018_3D import CenterCropBRATS3D, NormalizeBRATS3D
from models.resnet3D import resnet50_3D
from tqdm import tqdm

In [2]:
# helper functions
def time_stamp() -> str:
    ts = time.time()
    time_stamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    return time_stamp

In [3]:
def classification_infer(case_name, val_dir, model):
    case_dir = os.path.join(val_dir, case_name)
    
    t1 = nib.load(os.path.join(case_dir, case_name + '_t1.nii.gz')).get_data()
    t1ce = nib.load(os.path.join(case_dir, case_name + '_t1ce.nii.gz')).get_data()
    t2 = nib.load(os.path.join(case_dir, case_name + '_t2.nii.gz')).get_data()
    flair = nib.load(os.path.join(case_dir, case_name + '_flair.nii.gz')).get_data()
    
    assert t1.shape == (240, 240, 155)
    assert t1ce.shape == (240, 240, 155)
    assert t2.shape == (240, 240, 155)
    assert flair.shape == (240, 240, 155)
    
    sc = np.array([t1, t1ce, t2, flair])
    assert sc.shape == (4, 240, 240, 155)
    
    # fake label for position
    fake_label = np.array([1])
    
    center_crop = CenterCropBRATS3D()
    normalize = NormalizeBRATS3D()
    to_tensor = ToTensorVal()
    
    sc, fake_label = center_crop((sc, fake_label))
    sc, fake_label = normalize((sc, fake_label))
    sc = to_tensor(sc)
    
    # unsqueeze to 5-dimension array, NxCxHxWxD
    sc = torch.unsqueeze(sc, dim=0)
    sc = sc.to(device)
    
    with torch.no_grad():
        label = model(sc)
        label = torch.argmax(F.softmax(label, dim=1), dim=1, keepdim=True)
    
    # squeeze to rank-0 array
    label = torch.squeeze(label)
    label = label.cpu().numpy()
    
    return label

In [4]:
def infer(case_name, seg_type, model_path, device):
    case_dir = os.path.join(val_dir, case_name)
    if seg_type == 'et' or seg_type == 'tc':
        scan_path = os.path.join(case_dir, case_name + '_t1ce.nii.gz')
        sc = nib.load(scan_path)
        sc = sc.get_data()
        sc = sc.transpose((2, 0, 1))
        assert sc.shape == (155, 240, 240)
        
        model = UNet(n_channels=1, n_classes=1, residual=True)
    elif seg_type == 'wt':
        t2_path = os.path.join(case_dir, case_name + '_t2.nii.gz')
        flair_path = os.path.join(case_dir, case_name + '_flair.nii.gz')
        t2 = nib.load(t2_path).get_data()
        flair = nib.load(flair_path).get_data()
        sc = np.array([t2, flair]).transpose((3, 0, 1, 2))
        assert sc.shape == (155, 2, 240, 240)
        
        model = UNet(n_channels=2, n_classes=1, residual=True)
    elif seg_type == 'seg':
        t1_path = os.path.join(case_dir, case_name + '_t1.nii.gz')
        t1ce_path = os.path.join(case_dir, case_name + '_t1ce.nii.gz')
        t2_path = os.path.join(case_dir, case_name + '_t2.nii.gz')
        flair_path = os.path.join(case_dir, case_name + '_flair.nii.gz')
        
        t1 = nib.load(t1_path).get_data()
        t1ce = nib.load(t1ce_path).get_data()
        t2 = nib.load(t2_path).get_data()
        flair = nib.load(flair_path).get_data()
        sc = np.array([t1, t1ce, t2, flair]).transpose((3, 0, 1, 2))
        assert sc.shape == (155, 4, 240, 240)
        
        model = UNet(n_channels=4, n_classes=4, residual=True, expansion=2)
    else:
        raise ValueError('seg_type should only be et, tc, wt or seg')
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    
    preds = np.zeros((155, 240, 240))
    
    for i in range(155):
        slice_i = sc[i]
        if seg_type == 'et' or seg_type == 'tc':
            slice_i = np.expand_dims(slice_i, axis=0)
            assert slice_i.shape == (1, 240, 240)
        elif seg_type == 'wt':
            assert slice_i.shape == (2, 240, 240)
        elif seg_type == 'seg':
            assert slice_i.shape == (4, 240, 240)
        
        normalize = NormalizeBRATSVal()
        totensor = ToTensorVal()
        
        slice_i = normalize(slice_i)
        slice_i = totensor(slice_i)
        
        # unsqueeze the dimension to 4, NxCxHxW
        slice_i = torch.unsqueeze(slice_i, dim=0)
        slice_i = slice_i.to(device)
        with torch.no_grad():
            output = model(slice_i)
            if seg_type == 'seg':
                pred = torch.argmax(F.softmax(output, dim=1), dim=1, keepdim=True)
            else:
                pred = torch.sigmoid(output) > 0.5
        
        # squeeze the dimension down to 2, HxW
        pred = torch.squeeze(pred)
        pred = pred.cpu().numpy()
        preds[i] = pred
            
    
    return preds

# sorted validation cases

In [4]:
case_list = sorted(os.listdir('BRATS2018_Validation/'))
val_dir = 'BRATS2018_Validation/'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if not os.path.exists('validations/'):
    os.makedirs('validations/')

## do the inference for Wt, ET, and TC separately

In [5]:
for case_name in tqdm(case_list):
    preds_wt = infer(case_name, 'wt', '../UNetResidualBlock_DiceLoss_WT_210_manual/UNets-ResidualBlock-BRATS2018-210-WT_batch8_training_epochs15_Adam_scheduler-step10-gamma1.0_lr0.0001_w_decay1e-05/trained_model.pt',\
         device)
    assert preds_wt.shape == (155, 240, 240)
    print(f'{time_stamp()} Finish {case_name} whole tumor.')
    
    preds_et = infer(case_name, 'et', '../UNetResidualBlock_DiceLoss_ET_210_manual/UNet-Residual-Block-BRATS2018-210-ET_batch8_training_epochs10_Adam_scheduler-step10-gamma1.0_lr1e-05_w_decay1e-05/terminated_model.pt',\
         device)
    assert preds_et.shape == (155, 240, 240)
    print(f'{time_stamp()} Finish {case_name} enhancing tumor.')
    
    preds_tc = infer(case_name, 'tc', '../UNetResidualBlock_DiceLoss_TC_210_manual/UNet-ResidualBlock-BRATS2018-210-TC_batch8_training_epochs15_Adam_scheduler-step10-gamma1.0_lr3e-05_w_decay1e-05/terminated_model.pt',\
         device)
    assert preds_tc.shape == (155, 240, 240)
    print(f'{time_stamp()} Finish {case_name} tumor core.')
    
    preds_2 = np.logical_xor(preds_wt, preds_tc).astype(np.uint8)
    preds_1 = np.logical_xor(preds_tc, preds_et).astype(np.uint8)
    preds_4 = preds_et.astype(np.uint8)
    
    preds_seg = 4 * preds_4 + 2 * preds_2 + preds_1
    assert preds_seg.shape == (155, 240, 240)
    
    # get the mask of which computed label is greater than 4, and set it to 4
    preds_seg = preds_seg + (preds_seg > 4).astype(np.uint8) * (4 * np.ones((155, 240, 240)) - preds_seg)
    preds_seg = preds_seg + (preds_seg == 3) * np.ones((155, 240, 240))
    preds_seg = preds_seg.astype(np.uint8)
    print(np.unique(preds_seg))
    
    img = nib.Nifti1Image(preds_seg.transpose(1, 2, 0), affine=None)
    nib.save(img, os.path.join('validations', case_name + '.nii.gz'))
    
    

  0%|          | 0/66 [00:00<?, ?it/s]/home/martin/Documents/semantic_segmentation/PyTorch_venv/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


2019-05-03 23:19:10 Finish Brats18_CBICA_AAM_1 whole tumor.
2019-05-03 23:19:23 Finish Brats18_CBICA_AAM_1 enhancing tumor.
2019-05-03 23:19:36 Finish Brats18_CBICA_AAM_1 tumor core.


  2%|▏         | 1/66 [00:43<47:23, 43.75s/it]

[0 1 2 4]
2019-05-03 23:19:51 Finish Brats18_CBICA_ABT_1 whole tumor.
2019-05-03 23:20:04 Finish Brats18_CBICA_ABT_1 enhancing tumor.
2019-05-03 23:20:18 Finish Brats18_CBICA_ABT_1 tumor core.


  3%|▎         | 2/66 [01:25<45:55, 43.05s/it]

...

[0 1 2 4]
2019-05-04 00:03:52 Finish Brats18_WashU_W047_1 whole tumor.
2019-05-04 00:04:06 Finish Brats18_WashU_W047_1 enhancing tumor.
2019-05-04 00:04:19 Finish Brats18_WashU_W047_1 tumor core.


 98%|█████████▊| 65/66 [45:26<00:42, 42.12s/it]

[0 1 2 4]
2019-05-04 00:04:34 Finish Brats18_WashU_W053_1 whole tumor.
2019-05-04 00:04:48 Finish Brats18_WashU_W053_1 enhancing tumor.
2019-05-04 00:05:02 Finish Brats18_WashU_W053_1 tumor core.


100%|██████████| 66/66 [46:08<00:00, 42.17s/it]

[0 1 2 4]


## infer 4 classes in one forward pass

In [5]:
case_list = sorted(os.listdir('BRATS2018_Validation/'))
val_dir = 'BRATS2018_Validation'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cls_model = resnet50_3D(num_classes=2)
cls_model.load_state_dict(torch.load('../ResNet50-3D-32-CLS/trained_model.pt', map_location=device))
cls_model.to(device)
cls_model.eval()

ResNet3D(
  (conv1): Conv3d(4, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
  (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck3D(
      (conv1): Conv3d(32, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(32, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv3d(32, 128, kernel_

In [6]:
for case_name in tqdm(case_list):
    label = classification_infer(case_name, val_dir, cls_model)
    if label == 0.:
        print(f'{case_name} - LGG')
        preds = infer(case_name, 'seg', '../UNet-ResidualBlock-Expansion_LGG_end_to_end/epoch10_model.pt', device)
    else:
        print(f'{case_name} - HGG')
        preds = infer(case_name, 'seg', '../UNet-ResidualBlock-Expansion_210_end_to_end_manual/UNet-ResidualBlock-Expansion-BRATS2018-End-to-End_batch6_training_epochs15_Adam_scheduler-step10-gamma1.0_lr5e-05_w_decay3e-05/trained_model.pt',\
                 device)
    assert preds.shape == (155, 240, 240)
    print(f'{time_stamp()} Finish {case_name}.')
    
    # map label (0, 1, 2, 3) to (0, 1, 2, 4)
    preds = preds + (preds == 3).astype(np.uint8) * np.ones((155, 240, 240))
    preds = preds.astype(np.uint8)
    if np.sum((preds == 4).astype(np.uint8)) < 700:
        preds = preds + (preds == 4).astype(np.uint8) * (-4) * np.ones((155, 240, 240))
        print('enhancing tumor voxels are fewer than 700, mask out')
    print(np.unique(preds))
    
    img = nib.Nifti1Image(preds.transpose(1, 2, 0), affine=None)
    nib.save(img, os.path.join('validations', case_name + '.nii.gz'))

  0%|          | 0/66 [00:00<?, ?it/s]

Brats18_CBICA_AAM_1 - HGG


/home/martin/Documents/semantic_segmentation/PyTorch_venv/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


2019-05-15 11:41:11 Finish Brats18_CBICA_AAM_1.
[0 1 2 4]


  2%|▏         | 1/66 [00:23<25:08, 23.21s/it]

...

 97%|█████████▋| 64/66 [24:35<00:46, 23.25s/it]

Brats18_WashU_W047_1 - HGG
2019-05-15 12:05:46 Finish Brats18_WashU_W047_1.
[0 1 2 4]


 98%|█████████▊| 65/66 [24:58<00:23, 23.27s/it]

Brats18_WashU_W053_1 - HGG
2019-05-15 12:06:09 Finish Brats18_WashU_W053_1.
[0 1 2 4]


100%|██████████| 66/66 [25:21<00:00, 23.28s/it]


## inference for possible LGG cases

In [4]:
val_dir = 'BRATS2018_Validation/'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
lgg_case_list = ['Brats18_TCIA09_248_1', 'Brats18_TCIA10_195_1', 'Brats18_TCIA10_311_1',\
                'Brats18_TCIA10_609_1', 'Brats18_TCIA13_617_1',\
                'Brats18_TCIA13_638_1', 'Brats18_TCIA13_652_1', 'Brats18_UAB_3499_1']

In [5]:
for case_name in tqdm(lgg_case_list):
    preds = infer(case_name, 'seg', '../UNet-ResidualBlock-Expansion_LGG_end_to_end/epoch10_model.pt',\
                 device)
    assert preds.shape == (155, 240, 240)
    print(f'{time_stamp()} Finish {case_name}.')
    
    # map label (0, 1, 2, 3) to (0, 1, 2, 4)
    preds = preds + (preds == 3).astype(np.uint8) * np.ones((155, 240, 240))
    preds = preds.astype(np.uint8)
    print(np.unique(preds))
    
    img = nib.Nifti1Image(preds.transpose(1, 2, 0), affine=None)
    nib.save(img, os.path.join('../validations_lgg/', case_name + '.nii.gz'))

  0%|          | 0/8 [00:00<?, ?it/s]/home/martin/Documents/semantic_segmentation/PyTorch_venv/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


2019-05-14 19:52:16 Finish Brats18_TCIA09_248_1.


 12%|█▎        | 1/8 [00:27<03:10, 27.18s/it]

[0 1 2 4]
2019-05-14 19:52:39 Finish Brats18_TCIA10_195_1.
[0 1 2 4]


 25%|██▌       | 2/8 [00:49<02:34, 25.74s/it]

...

 75%|███████▌  | 6/8 [02:19<00:46, 23.19s/it]

2019-05-14 19:54:31 Finish Brats18_TCIA13_652_1.
[0 1 2 4]


 88%|████████▊ | 7/8 [02:41<00:22, 22.95s/it]

2019-05-14 19:54:54 Finish Brats18_UAB_3499_1.
[0 1 2 4]


100%|██████████| 8/8 [03:03<00:00, 22.79s/it]
